In [ ]:
from networkproperties import *
from creatingnetworks import *
from dynamicsfunctions import *
from bestfit import *
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


In [ ]:
def ensure_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    return Path(path)

def sem(x, axis=0):
    x = np.asarray(x, dtype=float)
    n = x.shape[axis]
    if n <= 1:
        return np.zeros_like(np.mean(x, axis=axis))
    return np.std(x, axis=axis, ddof=1) / np.sqrt(n)

def plateau_mean(v, tail=100):
    v = np.asarray(v, dtype=float)
    tail = min(int(tail), max(1, len(v)//2))
    return float(v[-tail:].mean())

def rel_diff_percent(a, b):
    a = float(a)
    b = float(b)
    return np.nan if a == 0 else abs((b - a) / a) * 100.0




In [ ]:
def run_ba_model1_vs_model3(
    N=1000,
    max_steps=500,
    sigma=0.15,
    num_runs=30,
    num_realizations=30,
    m_values=None,
    q_fixed=0.5,
    mu=0.0,
    p_i=1.0,
    p_e=1.0,
    tail=100,
    out_dir="./results_ba_m_compare",
    seed0=123,
):

    if m_values is None:
        m_values = [5,10,15,20,25,30,40,50,60,80,90,100,110,120,140,160,180,200,240,250]

    out_dir = ensure_dir(out_dir)
    rng = np.random.default_rng(seed0)

    ts_rows = []
    net_rows = []
    sum_rows = []

    for m in m_values:
        m_dir = ensure_dir(out_dir / f"m_{int(m)}")
        fig_dir = ensure_dir(m_dir / "figs")
        csv_dir = ensure_dir(m_dir / "csv")
        deg_dir = ensure_dir(m_dir / "degdist")

        per_real_mean_m1 = np.zeros((num_realizations, max_steps + 1), dtype=float)
        per_real_mean_m3 = np.zeros((num_realizations, max_steps + 1), dtype=float)
        per_real_runsem_m1 = np.zeros((num_realizations, max_steps + 1), dtype=float)
        per_real_runsem_m3 = np.zeros((num_realizations, max_steps + 1), dtype=float)

        pl_m1 = np.zeros(num_realizations, dtype=float)
        pl_m3 = np.zeros(num_realizations, dtype=float)
        rel_d = np.zeros(num_realizations, dtype=float)

        pk_list = []

        for r in range(num_realizations):
            seed = int(rng.integers(0, 2**31 - 1))
            G = nx.barabasi_albert_graph(N, int(m), seed=seed)
            if not nx.is_connected(G):
                for t in range(10):
                    G = nx.barabasi_albert_graph(N, int(m), seed=seed + t + 1)
                    if nx.is_connected(G):
                        break
            if not nx.is_connected(G):
                raise RuntimeError(f"BA disconnected (unexpected) for m={m}")

            N0, E0, k0, C0, L0, lam2 = structural_metrics(G)
            net_rows.append({
                "m": m, "realization": r,
                "N": N0, "E": E0, "k_mean": k0,
                "C": C0, "L": L0, "lambda2": lam2
            })

            _, degrees, flat, ptr = prepare_network_data(G)

            _, pk = degree_dist_from_degrees(degrees)
            pk_list.append(pk)

            runs_m1 = np.zeros((num_runs, max_steps + 1), dtype=float)
            runs_m3 = np.zeros((num_runs, max_steps + 1), dtype=float)

            for run_idx in range(num_runs):
                # Model 1: fixed q
                runs_m1[run_idx] = single_run_general(
                    flat, ptr, degrees,
                    max_steps, sigma, N, mu,
                    p_i, p_e,
                    q_fixed,
                    NEI_UNIFORM,
                    Q_FIXED
                )
                # Model 3: dynamic q
                runs_m3[run_idx] = single_run_general(
                    flat, ptr, degrees,
                    max_steps, sigma, N, mu,
                    p_i, p_e,
                    q_fixed,        
                    NEI_UNIFORM,
                    Q_DYNAMIC
                )

            per_real_mean_m1[r, :] = runs_m1.mean(axis=0)
            per_real_mean_m3[r, :] = runs_m3.mean(axis=0)
            per_real_runsem_m1[r, :] = sem(runs_m1, axis=0)
            per_real_runsem_m3[r, :] = sem(runs_m3, axis=0)

            pl_m1[r] = plateau_mean(per_real_mean_m1[r], tail=tail)
            pl_m3[r] = plateau_mean(per_real_mean_m3[r], tail=tail)
            rel_d[r] = rel_diff_percent(pl_m1[r], pl_m3[r])

        def aggregate(per_real_mean, per_real_runsem, R):
            mean = per_real_mean.mean(axis=0)

            var_M = per_real_mean.var(axis=0, ddof=1) if R > 1 else np.zeros_like(mean)

            within_var_mean = (per_real_runsem**2).mean(axis=0)

            sem_runs_component = np.sqrt(within_var_mean / R)
            sem_network        = np.sqrt(var_M / R)       
            sem_total          = np.sqrt((within_var_mean + var_M) / R)

            return mean, sem_total, sem_network, sem_runs_component

        mean_m1, semtot_m1, semnet_m1, semrun_m1 = aggregate(per_real_mean_m1, per_real_runsem_m1)
        mean_m3, semtot_m3, semnet_m3, semrun_m3 = aggregate(per_real_mean_m3, per_real_runsem_m3)

        df_net_m = pd.DataFrame([x for x in net_rows if x["m"] == m])
        def msem(col):
            if len(df_net_m) <= 1:
                return float(df_net_m[col].mean()), 0.0
            return float(df_net_m[col].mean()), float(df_net_m[col].std(ddof=1)/np.sqrt(len(df_net_m)))

        C_mean, C_sem = msem("C")
        L_mean, L_sem = msem("L")
        lam2_mean, lam2_sem = msem("lambda2")
        k_mean, k_sem = msem("k_mean")

        pl1_mean = float(np.mean(pl_m1))
        pl3_mean = float(np.mean(pl_m3))
        pl1_sem  = float(np.std(pl_m1, ddof=1)/np.sqrt(num_realizations)) if num_realizations > 1 else 0.0
        pl3_sem  = float(np.std(pl_m3, ddof=1)/np.sqrt(num_realizations)) if num_realizations > 1 else 0.0

        rd_mean = float(np.mean(rel_d))
        rd_sem  = float(np.std(rel_d, ddof=1)/np.sqrt(num_realizations)) if num_realizations > 1 else 0.0

        sum_rows.append({
            "m": m,
            "plateau_m1_mean": pl1_mean, "plateau_m1_sem": pl1_sem,
            "plateau_m3_mean": pl3_mean, "plateau_m3_sem": pl3_sem,
            "reldiff_mean_pct": rd_mean, "reldiff_sem_pct": rd_sem,
            "C_mean": C_mean, "C_sem": C_sem,
            "L_mean": L_mean, "L_sem": L_sem,
            "lambda2_mean": lam2_mean, "lambda2_sem": lam2_sem,
            "k_mean": k_mean, "k_sem": k_sem
        })

        tgrid = np.arange(max_steps + 1)
        for t in range(max_steps + 1):
            ts_rows.append({
                "m": m, "t": t, "model": "model1_fixed_q",
                "variance_mean": float(mean_m1[t]),
                "sem_total": float(semtot_m1[t]),
                "sem_network": float(semnet_m1[t]),
                "sem_runs": float(semrun_m1[t]),
            })
            ts_rows.append({
                "m": m, "t": t, "model": "model3_dynamic_q",
                "variance_mean": float(mean_m3[t]),
                "sem_total": float(semtot_m3[t]),
                "sem_network": float(semnet_m3[t]),
                "sem_runs": float(semrun_m3[t]),
            })

     
        plt.figure(figsize=(7.2,4.6), dpi=300)
        plt.plot(tgrid, mean_m1, label="Model 1 (fixed q)")
        plt.fill_between(tgrid, mean_m1 - semtot_m1, mean_m1 + semtot_m1, alpha=0.20)
        plt.plot(tgrid, mean_m3, label="Model 3 (dynamic q)")
        plt.fill_between(tgrid, mean_m3 - semtot_m3, mean_m3 + semtot_m3, alpha=0.20)
        plt.xlabel("Time step")
        plt.ylabel("Variance")
        plt.title(f"BA (m={m}): Model 1 vs Model 3")
        plt.legend()
        plt.tight_layout()
        plt.savefig(fig_dir / f"variance_models_m{m}.png")
        plt.close()

        if len(pk_list) > 0:
            H = pad_and_stack_pk(pk_list)
            pk_mean = H.mean(axis=0)
            pk_sem  = sem(H, axis=0)
            kk = np.arange(len(pk_mean))
            df_deg = pd.DataFrame({"deg": kk, "pk_mean": pk_mean, "pk_sem": pk_sem})
            df_deg.to_csv(deg_dir / f"degdist_BA_m{m}.csv", index=False)

            plt.figure(figsize=(6.0,4.0), dpi=300)
            plt.bar(df_deg["deg"], df_deg["pk_mean"], width=0.9, alpha=0.9)
            plt.errorbar(df_deg["deg"], df_deg["pk_mean"], yerr=df_deg["pk_sem"],
                         fmt="none", ecolor="gray", elinewidth=1, capsize=0)
            plt.xlabel("Degree")
            plt.ylabel("P(k)")
            plt.title(f"BA degree distribution (m={m})")
            plt.tight_layout()
            plt.savefig(deg_dir / f"degdist_BA_m{m}.png")
            plt.close()

        df_ts_m = pd.DataFrame([x for x in ts_rows if x["m"] == m])
        df_sum_m = pd.DataFrame([x for x in sum_rows if x["m"] == m])
        df_net_m.to_csv(csv_dir / f"network_metrics_per_realization_m{m}.csv", index=False)
        df_ts_m.to_csv(csv_dir / f"timeseries_long_m{m}.csv", index=False)
        df_sum_m.to_csv(csv_dir / f"summary_m{m}.csv", index=False)

    df_summary_m = pd.DataFrame(sum_rows).sort_values("m").reset_index(drop=True)
    df_ts_long = pd.DataFrame(ts_rows)
    df_net_real = pd.DataFrame(net_rows)

    df_summary_m.to_csv(out_dir / "summary_ALL_m.csv", index=False)
    df_ts_long.to_csv(out_dir / "timeseries_ALL_m.csv", index=False)
    df_net_real.to_csv(out_dir / "network_metrics_per_realization_ALL_m.csv", index=False)

    fig_global = ensure_dir(out_dir / "global_figs")

    #  plateau vs m for both models
    plt.figure(figsize=(7.2,4.6), dpi=300)
    plt.errorbar(df_summary_m["m"], df_summary_m["plateau_m1_mean"], yerr=df_summary_m["plateau_m1_sem"],
                 fmt="o-", capsize=0, label="Model 1 plateau")
    plt.errorbar(df_summary_m["m"], df_summary_m["plateau_m3_mean"], yerr=df_summary_m["plateau_m3_sem"],
                 fmt="s-", capsize=0, label="Model 3 plateau")
    plt.xlabel("BA parameter m")
    plt.ylabel("Plateau variance (tail mean)")
    plt.title("Plateau variance vs m")
    plt.legend()
    plt.tight_layout()
    plt.savefig(fig_global / "plateau_vs_m_models.png")
    plt.close()

    #  relative difference vs m
    plt.figure(figsize=(7.2,4.6), dpi=300)
    plt.errorbar(df_summary_m["m"], df_summary_m["reldiff_mean_pct"], yerr=df_summary_m["reldiff_sem_pct"],
                 fmt="o-", capsize=0)
    plt.xlabel("BA parameter m")
    plt.ylabel("Relative difference (%)")
    plt.title("Relative difference between Model 3 and Model 1 (plateau)")
    plt.tight_layout()
    plt.savefig(fig_global / "reldiff_vs_m.png")
    plt.close()

    # best-fit 
    m_arr = df_summary_m["m"].to_numpy(dtype=float)
    y_arr = df_summary_m["reldiff_mean_pct"].to_numpy(dtype=float)
    fit = fit_plateau_models(m_arr, y_arr)

    if fit is not None:
        plt.figure(figsize=(7.2,4.8), dpi=300)
        plt.scatter(m_arr, y_arr, s=35, alpha=0.9, label="data (mean)")
        plt.plot(fit["m_grid"], fit["y_grid"], lw=2.0,
                 label=f'best: {fit["best_name"]}  (AIC={fit["aic"]:.2f})')
        plt.xlabel("BA parameter m")
        plt.ylabel("Relative difference (%)")
        plt.title("Relative difference vs m: best fitted analytical form")
        plt.legend()
        plt.text(0.02, 0.95, fit["equation"], transform=plt.gca().transAxes,
                 va="top", ha="left")
        plt.tight_layout()
        plt.savefig(fig_global / "bestfit_reldiff_vs_m.png")
        plt.close()


        plt.figure(figsize=(7.2,4.8), dpi=300)
        mask = (m_arr > 0) & (y_arr > 0)
        plt.loglog(m_arr[mask], y_arr[mask], "o", alpha=0.9, label="data (mean)")
        maskg = (fit["m_grid"] > 0) & (fit["y_grid"] > 0)
        plt.loglog(fit["m_grid"][maskg], fit["y_grid"][maskg], "-", lw=2.0, label="best fit")
        plt.xlabel("m")
        plt.ylabel("Relative difference (%)")
        plt.title("Log–log view: relative difference vs m (best fit)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(fig_global / "bestfit_reldiff_vs_m_loglog.png")
        plt.close()

        fit_info = pd.DataFrame([{
            "best_model": fit["best_name"],
            "equation": fit["equation"],
            "aic": fit["aic"],
            "a": fit["params"][0],
            "b": fit["params"][1],
            "c": fit["params"][2],
        }])
        fit_info.to_csv(out_dir / "bestfit_reldiff_vs_m_info.csv", index=False)

    return df_summary_m, df_ts_long, df_net_real


In [6]:
df_sum_m, df_ts_m, df_net_m = run_ba_model1_vs_model3(
    N=1000,
    max_steps=500,
    sigma=0.15,
    num_runs=30,
    num_realizations=30,
    m_values=[5,10,15,20,25,30,40,50,60,80,100,120,125],
    q_fixed=0.5,
    mu=0.0,
    p_i=1.0,
    p_e=1.0,
    tail=100,
    out_dir="./results_ba_m_compare",
    seed0=123
)

df_sum_m.head()


,m,plateau_m1_mean,plateau_m1_sem,plateau_m3_mean,plateau_m3_sem,reldiff_mean_pct,reldiff_sem_pct,C_mean,C_sem,L_mean,L_sem,lambda2_mean,lambda2_sem,k_mean,k_sem
0,5,0.048978,0.000019,0.070462,0.000211,43.864571,0.431500,0.040741,0.000577,2.979380,0.002264,2.762895,0.088081,9.95,3.298612e-16
1,10,0.046640,0.000013,0.061293,0.000122,31.416592,0.261065,0.060619,0.000435,2.556357,0.000965,7.158001,0.123755,19.80,6.597224e-16
2,15,0.046028,0.000017,0.058204,0.000092,26.455521,0.192055,0.079394,0.000400,2.327979,0.000896,11.771908,0.243788,29.55,1.319445e-15
3,20,0.045703,0.000014,0.056437,0.000070,23.486823,0.161278,0.095547,0.000405,2.166309,0.000701,15.753000,0.732606,39.20,1.319445e-15
4,25,0.045504,0.000014,0.055214,0.000064,21.338601,0.147768,0.110186,0.000353,2.061763,0.000425,21.137641,0.558001,48.75,0.000000e+00
